In [1]:
import pandas as pd
import ee
from collections import defaultdict
import numpy as np
import datetime
ee.Initialize()
import random
import time

# Parameters

In [2]:
# GOOGLE DRIVE FOLDER
GDRIVE_FOLDER = 'wfp-allbands-unlabeled-b1'

# 13 bands or RGB?
ALL_BANDS = True



# NUMBER OF FILES TO DOWNLOAD
DOWNLOAD_NB_FILES = 1000

# RADIUS AROUND COORD IN METERS
# This is the number of meter around the point coordinate to include in the picture
RADIUS_AROUND = 400


# RANGES FOR BANDS
# Values tested on different location in nepal
# Hypothesis: Should be the same for all images
RANGE_MIN = 0
RANGE_MAX = 2000


# RANGE FOR DATES
# we need to take several pictures to select ones without clouds
# We look for RANDE_DATE weeks around the date
RANDE_DATE = 2

# TIME OF INTEREST
# A random date is selected between TOI_BEGIN and TOI_END
TOI_BEGIN = '01/01/2016'
TOI_END = '01/06/2019'

# MAXIMUM % OF CLOUDS
MAX_CLOUDS = 10

# LAUNCH EXPORT
LAUNCH_EXPORT = True

# REGION TO TAKE PICTURE FROM
NEPAL = ee.Geometry.Polygon([[84.46716739280828,29.108665503908366],
[82.84929902963563,29.95080069391136],
[81.9899776971655,30.413866129203402],
[81.01575580840813,30.491203262843925],
[79.96277286155828,28.849173773303857],
[82.82709294386302,27.405490759555242],
[83.79152604893773,27.280548634789366],
[84.12517363514314,27.354698207754293],
[84.61046405920763,27.23515863534531],
[84.56770354968148,26.99802099337535],
[86.30188418968328,26.48893917833053],
[86.80725528343328,28.128845548855082],
[85.631313693822,28.638952723497773],
[84.46716739280828,29.108665503908366]])

# Google Earth Engine download


In [3]:
# HELPER FUNCTIONS

def stringify_date(date):
    #return f'{date.day}/{date.month}/{date.year}'
    
    return date.strftime("%d/%m/%Y")

def get_name_im(x, y, date):
    return f"{str(x)[:9].replace('.', '-')}_{str(y)[:9].replace('.', '-')}_{date}"

def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())),
    )
def get_date(data_str):
    d, m, y = data_str.split('/')
    
    return datetime.date(int(y), int(m), int(d))

def get_random_coordinate(geometry):
    rd_points = ee.FeatureCollection.randomPoints(geometry,1, random.randint(0, 1e10))
    
    return rd_points.getInfo()['features'][0]['geometry']['coordinates']

# Generate a rectangle containing the circle (centered on the coordinate) with radius RADIUS_AROUND
def get_geometry_radius(geometry_point):
    coord = np.array(geometry_point.getInfo()['coordinates'][0])
    return ee.Geometry.Rectangle([coord[:, 0].min(), coord[:, 1].min(), coord[:, 0].max(), coord[:, 1].max()])

# Generate the dates around the observation date
def date_range_to_collect(input_date, debug=False):
    d, m, y = input_date.split('/')
        
    target_date = datetime.date(int(y), int(m), int(d))
    delta = datetime.timedelta(weeks=RANDE_DATE)
    return target_date-delta, target_date+delta

In [4]:
def generate_images_random(image_collection, debug=True):
  

        start = get_date(TOI_BEGIN)
        end = get_date(TOI_END)
        
        date = random_date(start, end)
        
        x,y = get_random_coordinate(NEPAL)
        
        generate_image(image_collection, x, y, stringify_date(date), get_name_im(x,y,date), debug)


def generate_image(image_collection, x, y, date, image_name, debug=True):
    if debug: print(f'Working on {image_name}: ({x}, {y}) on {date}')
    geo = ee.Geometry.Point(x, y);
    radius = geo.buffer(RADIUS_AROUND);
    geometry_radius= get_geometry_radius(radius)
    
    spatialFiltered = image_collection.filterBounds(geo)

    date_range = date_range_to_collect(date, debug)
    if debug: print('date range:' + str(date_range[0]) + str(date_range[1]))
    temporalFiltered = spatialFiltered.filterDate(str(date_range[0]), str(date_range[1]))


    least_clouds = temporalFiltered.sort('CLOUD_PIXEL_PERCENTAGE').first()



    if ALL_BANDS:
        testimg = least_clouds.select('B.+')
    else:
        testimg = least_clouds.visualize(bands=['B4', 'B3', 'B2'], min=RANGE_MIN, max=RANGE_MAX)
        
    if LAUNCH_EXPORT:
        try:
            task = ee.batch.Export.image.toDrive(testimg, folder=GDRIVE_FOLDER, region=geometry_radius.getInfo()['coordinates'][0], description=image_name, scale=10)
            task.start()
        except:
            print('EXCEPTION DURING DOWNLOAD => TASK LIST SHOULD BE FULL. SLEEP FOR  2 hours')
            time.sleep(2*60*60)
            task = ee.batch.Export.image.toDrive(testimg, folder=GDRIVE_FOLDER, region=geometry_radius.getInfo()['coordinates'][0], description=image_name, scale=10)
            task.start()

In [5]:
dataset = ee.ImageCollection('COPERNICUS/S2').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than',MAX_CLOUDS)

for i, point in enumerate(range(DOWNLOAD_NB_FILES)):
    generate_images_random(dataset)

Working on 82-325205_29-611446_2017-05-02: (82.32520544714406, 29.611446897258325) on 02/05/2017
date range:2017-04-182017-05-16
Working on 86-550035_27-525978_2016-12-09: (86.55003585782869, 27.52597854604702) on 09/12/2016
date range:2016-11-252016-12-23
Working on 81-763558_28-936111_2019-01-24: (81.76355859188122, 28.93611123383822) on 24/01/2019
date range:2019-01-102019-02-07
Working on 82-085924_28-139609_2017-05-18: (82.08592495683769, 28.13960974651511) on 18/05/2017
date range:2017-05-042017-06-01
Working on 85-090526_28-211527_2017-02-02: (85.09052645457619, 28.211527927058505) on 02/02/2017
date range:2017-01-192017-02-16
Working on 86-248993_27-962654_2016-05-02: (86.24899336019033, 27.96265463184058) on 02/05/2016
date range:2016-04-182016-05-16
Working on 82-528776_29-850461_2017-05-05: (82.52877660322235, 29.85046106805893) on 05/05/2017
date range:2017-04-212017-05-19
Working on 83-490977_28-800867_2016-06-13: (83.49097753328559, 28.800867682163886) on 13/06/2016
date 

Working on 83-772087_28-582031_2019-05-18: (83.77208721493754, 28.58203114227346) on 18/05/2019
date range:2019-05-042019-06-01
Working on 84-002359_28-885615_2017-10-17: (84.00235981528128, 28.88561502903814) on 17/10/2017
date range:2017-10-032017-10-31
Working on 85-347150_27-234266_2018-01-20: (85.34715042828694, 27.234266595506014) on 20/01/2018
date range:2018-01-062018-02-03
Working on 82-093567_27-849998_2017-08-16: (82.0935675383014, 27.849998997150717) on 16/08/2017
date range:2017-08-022017-08-30
Working on 81-443516_29-851084_2017-07-07: (81.44351608686001, 29.85108424582796) on 07/07/2017
date range:2017-06-232017-07-21
Working on 86-047624_26-653117_2017-12-29: (86.04762466347589, 26.653117143818346) on 29/12/2017
date range:2017-12-152018-01-12
Working on 86-397891_27-551571_2017-10-05: (86.39789157056639, 27.551571392710233) on 05/10/2017
date range:2017-09-212017-10-19
Working on 83-984588_28-789955_2019-04-25: (83.984588075089, 28.789955316689092) on 25/04/2019
date r

Working on 81-158883_28-647162_2019-04-19: (81.15888336204013, 28.647162712071136) on 19/04/2019
date range:2019-04-052019-05-03
Working on 82-417338_27-818393_2016-05-26: (82.4173381538038, 27.818393883109596) on 26/05/2016
date range:2016-05-122016-06-09
Working on 85-942354_27-030998_2019-01-22: (85.94235497271644, 27.03099878509085) on 22/01/2019
date range:2019-01-082019-02-05
Working on 83-727446_28-378115_2017-01-19: (83.727446311546, 28.378115301930606) on 19/01/2017
date range:2017-01-052017-02-02
Working on 81-711678_29-920645_2016-01-01: (81.7116782533368, 29.920645852044654) on 01/01/2016
date range:2015-12-182016-01-15
Working on 81-346406_28-671080_2017-05-11: (81.34640604505658, 28.67108092235302) on 11/05/2017
date range:2017-04-272017-05-25
Working on 82-294218_29-143872_2019-04-26: (82.2942184653922, 29.14387285021556) on 26/04/2019
date range:2019-04-122019-05-10
Working on 81-752259_29-839208_2018-10-12: (81.7522596361441, 29.839208757008578) on 12/10/2018
date rang

Working on 82-580458_28-087735_2016-05-13: (82.58045816321938, 28.087735314883343) on 13/05/2016
date range:2016-04-292016-05-27
Working on 86-208491_27-484014_2016-09-14: (86.20849144923848, 27.48401455866208) on 14/09/2016
date range:2016-08-312016-09-28
Working on 82-894015_28-304689_2018-09-01: (82.89401569492368, 28.304689711558318) on 01/09/2018
date range:2018-08-182018-09-15
Working on 82-793399_29-439592_2018-06-17: (82.7933994084714, 29.439592739616117) on 17/06/2018
date range:2018-06-032018-07-01
Working on 86-335291_26-995978_2016-10-07: (86.33529133312597, 26.995978003965796) on 07/10/2016
date range:2016-09-232016-10-21
Working on 85-233898_27-864401_2018-01-23: (85.23389842428148, 27.864401799610906) on 23/01/2018
date range:2018-01-092018-02-06
Working on 85-879214_28-214763_2017-12-01: (85.87921442085698, 28.214763015184108) on 01/12/2017
date range:2017-11-172017-12-15
Working on 81-803436_29-400327_2017-11-09: (81.80343680936113, 29.400327932171056) on 09/11/2017
da

date range:2018-07-312018-08-28
Working on 85-313961_27-171198_2019-03-24: (85.31396136967363, 27.171198332938854) on 24/03/2019
date range:2019-03-102019-04-07
Working on 80-446450_29-000349_2018-04-08: (80.44645054288688, 29.00034967047895) on 08/04/2018
date range:2018-03-252018-04-22
Working on 82-059243_27-971299_2018-10-03: (82.05924347426048, 27.971299061004956) on 03/10/2018
date range:2018-09-192018-10-17
Working on 83-794129_28-659466_2016-11-05: (83.79412958362153, 28.65946660879264) on 05/11/2016
date range:2016-10-222016-11-19
Working on 83-664587_28-228959_2018-05-22: (83.66458724019436, 28.22895928814609) on 22/05/2018
date range:2018-05-082018-06-05
Working on 81-955113_30-208276_2018-02-19: (81.95511377061405, 30.20827616418165) on 19/02/2018
date range:2018-02-052018-03-05
Working on 83-194435_29-004526_2016-10-27: (83.19443529287453, 29.004526201297647) on 27/10/2016
date range:2016-10-132016-11-10
Working on 82-850815_27-954978_2017-10-15: (82.85081556062538, 27.954

...

Working on 82-335256_28-062236_2018-05-21: (82.33525652550996, 28.06223680256893) on 21/05/2018
date range:2018-05-072018-06-04
Working on 86-441492_28-156151_2018-07-12: (86.44149244390145, 28.15615113954932) on 12/07/2018
date range:2018-06-282018-07-26
Working on 86-269537_27-956483_2016-10-12: (86.26953744331482, 27.95648301419228) on 12/10/2016
date range:2016-09-282016-10-26
Working on 81-147104_29-506330_2018-08-02: (81.14710451998756, 29.506330835347) on 02/08/2018
date range:2018-07-192018-08-16
Working on 81-168673_29-884512_2016-12-11: (81.1686737808412, 29.88451236007514) on 11/12/2016
date range:2016-11-272016-12-25
Working on 85-676467_27-389797_2016-10-09: (85.67646731207041, 27.389797508435063) on 09/10/2016
date range:2016-09-252016-10-23
Working on 85-745591_28-143299_2017-08-12: (85.74559132132377, 28.143299625278495) on 12/08/2017
date range:2017-07-292017-08-26
Working on 82-733646_27-625271_2018-06-14: (82.73364625596754, 27.62527139030887) on 14/06/2018
date rang

Working on 86-194558_27-088208_2017-12-13: (86.1945589836884, 27.088208532207236) on 13/12/2017
date range:2017-11-292017-12-27
Working on 85-377713_28-197723_2018-05-21: (85.37771311379647, 28.197723625736206) on 21/05/2018
date range:2018-05-072018-06-04
Working on 83-198555_28-521829_2016-05-25: (83.1985557924492, 28.521829155233963) on 25/05/2016
date range:2016-05-112016-06-08
Working on 85-847563_27-046121_2018-03-09: (85.8475636778423, 27.046121568707957) on 09/03/2018
date range:2018-02-232018-03-23
Working on 86-364279_27-315790_2017-02-12: (86.36427993655762, 27.315790864828053) on 12/02/2017
date range:2017-01-292017-02-26
Working on 82-606619_28-727231_2018-08-06: (82.6066193516191, 28.72723154202415) on 06/08/2018
date range:2018-07-232018-08-20
Working on 83-070763_28-638038_2018-12-27: (83.07076344535344, 28.63803811914473) on 27/12/2018
date range:2018-12-132019-01-10
Working on 81-731765_28-245040_2019-05-18: (81.7317653035077, 28.245040812608707) on 18/05/2019
date ra

Working on 80-981989_28-858933_2018-03-12: (80.98198960890981, 28.8589330851121) on 12/03/2018
date range:2018-02-262018-03-26
Working on 81-276781_29-184935_2018-07-21: (81.27678146058537, 29.184935888909184) on 21/07/2018
date range:2018-07-072018-08-04
Working on 86-315735_27-131058_2017-03-29: (86.31573514086163, 27.131058403152675) on 29/03/2017
date range:2017-03-152017-04-12
Working on 84-822372_28-571496_2018-01-31: (84.82237217715415, 28.571496731016104) on 31/01/2018
date range:2018-01-172018-02-14
Working on 85-917984_28-333211_2017-02-05: (85.91798408621688, 28.333211392014366) on 05/02/2017
date range:2017-01-222017-02-19
Working on 83-853857_29-321850_2016-12-13: (83.85385746146942, 29.321850908430473) on 13/12/2016
date range:2016-11-292016-12-27
Working on 86-263035_28-149438_2018-07-21: (86.26303566822148, 28.149438246959637) on 21/07/2018
date range:2018-07-072018-08-04
Working on 80-698184_29-790406_2017-09-02: (80.69818494213072, 29.790406064399573) on 02/09/2017
da

Working on 80-787229_29-272884_2018-02-20: (80.78722984821434, 29.272884074729387) on 20/02/2018
date range:2018-02-062018-03-06
Working on 84-237265_27-563440_2018-02-14: (84.23726593709321, 27.56344029040428) on 14/02/2018
date range:2018-01-312018-02-28
Working on 81-713408_28-601881_2018-08-13: (81.71340821106409, 28.601881034938415) on 13/08/2018
date range:2018-07-302018-08-27
Working on 81-826631_29-851628_2017-03-03: (81.82663194934818, 29.851628504576947) on 03/03/2017
date range:2017-02-172017-03-17
Working on 84-789621_28-188310_2018-05-23: (84.78962140849315, 28.188310886389967) on 23/05/2018
date range:2018-05-092018-06-06
Working on 80-256117_29-018520_2016-01-19: (80.2561174424014, 29.018520779706577) on 19/01/2016
date range:2016-01-052016-02-02
Working on 83-396320_27-609131_2017-05-25: (83.39632034225511, 27.60913108094283) on 25/05/2017
date range:2017-05-112017-06-08
Working on 82-159486_29-817862_2018-02-08: (82.15948611283176, 29.817862635617082) on 08/02/2018
dat

Working on 85-084182_28-820523_2018-02-28: (85.08418288896536, 28.820523348847093) on 28/02/2018
date range:2018-02-142018-03-14
Working on 82-861846_29-685277_2017-12-28: (82.86184644799452, 29.685277172359665) on 28/12/2017
date range:2017-12-142018-01-11
Working on 83-455011_29-167061_2018-02-19: (83.45501127534054, 29.167061922893446) on 19/02/2018
date range:2018-02-052018-03-05
Working on 86-178114_26-836953_2017-04-02: (86.17811426062944, 26.836953187211613) on 02/04/2017
date range:2017-03-192017-04-16
Working on 83-188269_27-620679_2016-03-11: (83.18826999108725, 27.620679887368222) on 11/03/2016
date range:2016-02-262016-03-25
Working on 83-609495_27-439805_2016-06-06: (83.60949586296974, 27.439805102680296) on 06/06/2016
date range:2016-05-232016-06-20
Working on 86-443277_26-973482_2016-05-21: (86.44327762333681, 26.9734825218666) on 21/05/2016
date range:2016-05-072016-06-04
Working on 85-616945_28-075526_2017-08-15: (85.61694550284383, 28.075526311348085) on 15/08/2017
da